# Scrape ESPN Fantasy Football League

In order to scrape data from a private ESPN Fantasy Football League, we need to pass the SWID and the espn_s2 cookies values to the cookies parameter of the requests.get command.

In Chrome the SWID and the espn_s2 cookies values can be found here, chrome://settings/cookies/detail?site=espn.com.

To manually find the cookies values follow these instructions, settings > Cookies and other site data > All cookies and site data > espn.com > SWID/espn_s2.

Please note the cookies values differ from machine to machine.  My cookies values won't work on another comptuer.

<img src="SWID_cookie.PNG">

<img src="espn_s2_cookie.PNG">

Additionally, we need to pass the league id into the url.

To find the league id look in the url while logged into a private ESPN Fantasy Football league.

<img src="league_id.PNG">

To review any 2020 matchup:

Replace teamId with the team id value (can be found below); 

Replace matchupPeriodId and scoringPeriodId with the week number if it's a non-playoff matchup.  If it's a playoff matchup then matchupPeriodId and scoringPeriodId values are different.  For example, if we're looking for NFL weeks 15 and 16 which correspond to the last 2 playoff games in our league then matchupPeriodId = 14 and scoringPeriodId = 15 or 16 for NFL weeks 15 or 16.

https://fantasy.espn.com/football/boxscore?leagueId=169073&matchupPeriodId=1&scoringPeriodId=1&seasonId=2020&teamId=10&view=scoringperiod

In [1]:
# import needed packages
import numpy as np
import pandas as pd
import json
import os
import re, requests, bs4, csv, datetime
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# set pandas display options
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)
pd.set_option('display.width',1000)

In [ ]:
# set cookie values, league id, season, and current week number
# source: https://stmorse.github.io/journal/espn-fantasy-v3.html
swid      = '{F3E3A4D2-1203-4B11-BDAD-00C92AAA3F48}'
espn_s2   = 'AECf2TISP%2BRluNYIj24%2FqkuJBHTzAa7Br0BVIoUnBhHQCygnkN7hvyvU8wIm6XAT58otHnOZW4HjyImLJ14Rk8L9%2BwObWXeIa8kCosNFNMSc79r24KSiJK9jtqUQ2V1lvIncFM1qDhV9xL7E5jnutCP9rZfiJv8h%2Bq6WNvFb4YmyHICgx3QW68obk3wqBrGrXmw0vbq2bf367%2BsuL%2BOJ2ioRnxi1yY7LgbtiJnAXSKZj7kdIGjBiCPjUHgeXzDLHPunkLchEGlOrr%2FhG1ORgOsUX'
league_id = 169073
season    = 2017
week      = 17

# create url.  there are multiple views we can look at but we're interested in matchup data
url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
      str(season) + '/segments/0/leagues/' + str(league_id) + \
      '?view=mMatchup&view=mMatchupScore'

print(url)

# create JSON file for each week's matchup data
for i in range(1, week + 1):
    print('season: {}, week: {}'.format(season, i))
    r = requests.get(url,
                     params = {'scoringPeriodId': i},
                     cookies = {"SWID": swid, "espn_s2": espn_s2})
    d = r.json()
    with open('{}_matchups_week_{}.json'.format(season, i), 'w', encoding = 'utf-8') as f:
        json.dump(d, f, ensure_ascii = False, indent = 4)

In [2]:
matchups_list = []

In [5]:
# open JSON files and save returned dictionaries to a list
year = 2020
week = 17
#matchups_list = []

for i in range(1, week + 1):
    f = open('matchup_data/{}_matchups_week_{}.json'.format(year, i))
    
    # returns JSON object as a dictionary 
    data = json.load(f)
    matchups_list.append(data)

#### Create Dictionaries for Owner Names, Lineup Slot Names, Position Names, and NFL Team Names

In [6]:
# create dictionary of owner ids, owner team names, and owner names
owner_team_codes = {1:  ['Happy Rock Homewreckers', 'Blainer'],
                    2:  ["Bench Don't Kill My Vibe", 'Padge'],
                    4:  ['Seattle rainier riot', 'Boob'],
                    6:  ['Sticky Icky', 'T-$'],
                    7:  ['Springfield Atoms', 'Duvi'],
                    8:  ['Beacon Hill Posterizers', 'Bup'],
                    9:  ['Brookside Shokunin', 'Cheese'], 
                    10: ['CoMo FightinCamlToes', 'Doisy'],
                    11: ['Pixel Whippers','Sembower'],
                    15: ['Bud Lathrop Drive', 'Farmer']
}

# create dictionary of lineup slot ids and lineup names
lineup_slot_codes = {0:  'QB',
                     2:  'RB',
                     3:  'Flex',
                     4:  'WR',
                     6:  'TE', 
                     16: 'Def', 
                     17: 'K',
                     20: 'Bench', 
                     21: 'IR'
}

# create dictionary of position ids and position names
position_codes = {1:  'QB',
                  2:  'RB',
                  3:  'WR',
                  4:  'TE',
                  5:  'KR',
                  16: 'DEF'
    
}

# create dictionary of team ids and team names
pro_team_codes = {0:  ['Free Agent', np.nan],
                  1:  ['Atlanta Falcons', 'ATL'],
                  2:  ['Buffalo Bills', 'BUF'],
                  3:  ['Chicago Bears', 'CHI'],
                  4:  ['Cincinnati Bengals', 'CIN'],
                  5:  ['Cleveland Browns', 'CLE'],
                  6:  ['Dallas Cowboys', 'DAL'],
                  7:  ['Denver Broncs', 'DEN'],
                  8:  ['Detroit Lions', 'DET'],
                  9:  ['Greenbay Packers', 'GB'],
                  10: ['Tennessee Titans', 'TEN'],
                  11: ['Indianapolis Colts', 'IND'],
                  12: ['Kansas City Chiefs', 'KC'],
                  13: ['Las Vegas Raiders', 'LV'],
                  14: ['Los Angeles Rams', 'LA'],
                  15: ['Miami Dolphins', 'MIA'],
                  16: ['Minnesota Vikings', 'MIN'],
                  17: ['New Engalnd Patriots', 'NE'],
                  18: ['New Orleans Saints', 'NO'],
                  19: ['New York Giants', 'NYG'],
                  20: ['New York Jets', 'NYJ'],
                  21: ['Philadelphia Eagles', 'PHI'],
                  22: ['Arizona Cardinals', 'ARI'],
                  23: ['Pittsburgh Steelers', 'PIT'],
                  24: ['Los Angeles Chargers', 'LAC'],
                  25: ['San Francisco 49ers', 'SF'],
                  26: ['Seattle Seahawks', 'SEA'],
                  27: ['Tampa Bay Buccaneers', 'TB'],
                  28: ['Washington Football Team', 'WAS'],
                  29: ['Carolina Panthers', 'CAR'],
                  30: ['Jacksonville Jaguars', 'JAX'],
                  33: ['Baltimore Ravens', 'BAL'],
                  34: ['Houston Texans', 'HOU']
}

# create dictionary of fantasy scoring codes and fantasy scoring field names
# source: https://github.com/mkreiser/ESPN-Fantasy-Football-API/blob/master/src/player-stats/player-stats.js

### need codes for: ###
### fumble recovered for td; as opposed to fumble return for TD? ###
### FG 60+ yards


player_stat_codes = {0:   'pass_att',
                     1:   'pass_comp',
                     2:   'pass_incomp',
                     3:   'pass_yrd',
                     4:   'pass_td',
                     5:   'pass_5_yrd',
                     6:   'unk6',
                     7:   'unk7',
                     8:   'unk8',
                     9:   'unk9',
                     10:  'unk10',
                     11:  'unk11',
                     12:  'unk12',
                     13:  'unk13',
                     14:  'unk14',
                     15:  'unk15',
                     16:  'pass_50_yrd_td',
                     17:  'pass_yrd_300_399',
                     18:  'pass_yrd_400+',
                     19:  'unk19',
                     19:  'pass_2pt_con',
                     20:  'pass_int',
                     21:  'unk21',
                     22:  'pass_yrd_dupe',
                     23:  'rush_att',
                     24:  'rush_yrd',
                     25:  'rush_td',
                     26:  'rush_2pt_con',
                     27:  'rush_5_yrd',
                     28:  'unk28',
                     29:  'unk29',
                     30:  'unk30',
                     31:  'unk31',
                     32:  'unk32',
                     33:  'unk33',
                     34:  'unk34',
                     35:  'unk35',
                     36:  'rush_50_yrd_td',
                     37:  'rush_yrd_100_199',
                     38:  'rush_yrd_200+',
                     39:  'unk39',
                     40:  'unk40',
                     41:  'receptions_dupe',
                     42:  'rec_yrd',
                     43:  'rec_td',
                     44:  'rec_2pt_con',
                     45:  'unk45',
                     46:  'rec_50_yrd_td',
                     47:  'rec_5_yrd',
                     48:  'unk48',
                     49:  'unk49',
                     50:  'unk50',
                     51:  'unk51',
                     52:  'unk52',
                     53:  'receptions',
                     54:  'unk54',
                     55:  'unk55',
                     56:  'rec_yrd_100_199',
                     57:  'rec_yrd_200+',
                     58:  'rec_tar',
                     59:  'yac',
                     60:  'yrd_per_rec',
                     61:  'rec_yrd_dupe',
                     62:  'unk62',
                     64:  'unk64',
                     65:  'unk65',
                     66:  'unk66',
                     67:  'unk67',
                     68:  'unk68',
                     69:  'unk69',
                     70:  'unk70',
                     71:  'unk71',
                     72:  'fum_lost',
                     73:  'unk73',
                     74:  'fg_made_50+',
                     75:  'unk75',
                     76:  'unk76',
                     77:  'fg_made_40_49',
                     78:  'unk78',
                     79:  'fg_miss_40_49',
                     80:  'fg_made_0_39',
                     81:  'unk81',
                     82:  'fg_miss_0_39',
                     83:  'fg_con',
                     84:  'fg_att',  
                     85:  'fg_miss_tot',
                     86:  'pat_con',
                     87:  'pat_att',
                     88:  'pat_miss_tot',
                     89:  'def_st_0_pts_alw',
                     90:  'def_st_1_6_pts_alw',
                     91:  'def_st_7_13_pts_alw',
                     92:  'def_st_14_17_pts_alw',
                     93:  'def_st_blk_td',
                     94:  'unk94',
                     95:  'def_st_int',
                     96:  'def_st_fum',
                     97:  'def_st_blk_kick',
                     98:  'def_st_safety',
                     99:  'def_st_sack',
                     100: 'unk100',
                     101: 'def_st_kick_ret_td',
                     102: 'def_st_punt_ret_td',
                     103: 'def_st_int_td',
                     104: 'def_st_fum_ret_td',
                     105: 'unk105',
                     106: 'unk106',
                     107: 'unk107',
                     108: 'unk108',
                     109: 'unk109',
                     110: 'unk110',
                     111: 'unk111',
                     112: 'unk112',
                     113: 'unk113',
                     114: 'unk114',
                     115: 'unk115',
                     116: 'unk116',
                     117: 'unk117',
                     118: 'unk118',
                     119: 'unk119',
                     120: 'def_pts_alw',
                     121: 'unk121',
                     122: 'def_st_22_27_pts_alw',
                     123: 'def_st_28_34_pts_alw',
                     124: 'def_st_35_45_pts_alw',
                     125: 'def_st_46+_pts_alw',
                     127: 'def_tot_yrd_alw',
                     128: 'def_st_0_99_yrd_alw',
                     129: 'def_st_100_199_yrd_alw',
                     130: 'def_st_200_299_yrd_alw',
                     131: 'unk131',
                     132: 'def_st_350_399_yrd_alw',
                     133: 'def_st_400_449_yrd_alw',
                     134: 'def_st_450_499_yrd_alw',
                     135: 'def_st_500_549_yrd_alw',
                     136: 'def_st_550+_yrd_alw',
                     155: 'unk155',
                     156: 'unk156',
                     158: 'unk158',
                     175: 'unk175',
                     176: 'unk176',
                     177: 'unk177',
                     178: 'unk178',
                     179: 'unk179',
                     180: 'unk180',
                     181: 'unk181',
                     182: 'unk182',
                     183: 'unk183',
                     184: 'unk184',
                     185: 'unk185',
                     186: 'unk186',
                     187: 'unk187',
                     188: 'unk188',
                     189: 'unk189',
                     190: 'unk190',
                     191: 'unk191',
                     192: 'unk192',
                     193: 'unk193',
                     194: 'unk194',
                     195: 'unk195',
                     196: 'unk196',
                     197: 'unk197',
                     198: 'fg_made_50_59',
                     199: 'unk199',
                     200: 'unk200',
                     202: 'unk202',
                     203: 'unk203',
                     210: 'unk210',
}

ff_scoring_codes = {1:   'pass_comp_ff',
                    2:   'pass_incomp_ff',
                    4:   'pass_td_ff',
                    5:   'pass_5_yrd_ff',
                    16:  'pass_50_yrd_td_ff',
                    17:  'pass_yrd_300_399_ff',
                    18:  'pass_yrd_400+_ff',
                    19:  'pass_2pt_con_ff',
                    20:  'pass_int_ff',
                    25:  'rush_td_ff',
                    26:  'rush_2pt_con_ff',
                    27:  'rush_5_yrd_ff',
                    36:  'rush_50_yrd_td_ff',
                    37:  'rush_yrd_100_199_ff',
                    38:  'rush_yrd_200+_ff',
                    43:  'rec_td_ff',
                    44:  'rec_2pt_con_ff_ff',
                    46:  'rec_50_yrd_td_ff',
                    47:  'rec_5_yrd_ff',
                    53:  'receptions_ff',
                    56:  'rec_yrd_100_199_ff',
                    57:  'rec_yrd_200+_ff',
                    72:  'fum_lost_ff',
                    77:  'fg_made_40_49_ff',
                    79:  'fg_miss_40_49_ff',
                    80:  'fg_made_0_39_ff',
                    82:  'fg_miss_0_39_ff',
                    86:  'pat_made_ff',
                    88:  'pat_miss_ff',
                    89:  'def_st_0_pts_alw_ff',
                    90:  'def_st_1_6_pts_alw_ff',
                    91:  'def_st_7_13_pts_alw_ff',
                    92:  'def_st_14_17_pts_alw_ff',
                    93:  'def_st_blk_td_ff',
                    95:  'def_st_int_ff',
                    96:  'def_st_fum_ff',
                    97:  'def_st_blk_kick_ff',
                    98:  'def_st_safety_ff',
                    99:  'def_st_sack_ff',
                    101: 'def_st_kick_ret_td_ff',
                    102: 'def_st_punt_ret_td_ff',
                    103: 'def_st_int_td_ff',
                    104: 'def_st_fum_ret_td_ff',
                    122: 'def_st_22_27_pts_alw_ff',
                    123: 'def_st_28_34_pts_alw_ff',
                    124: 'def_st_35_45_pts_alw_ff',
                    125: 'def_st_46+_pts_alw_ff',
                    128: 'def_st_0_99_yrd_alw_ff',
                    129: 'def_st_100_199_yrd_alw_ff',
                    130: 'def_st_200_299_yrd_alw_ff',
                    132: 'def_st_350_399_yrd_alw_ff',
                    133: 'def_st_400_449_yrd_alw_ff',
                    134: 'def_st_450_499_yrd_alw_ff',
                    135: 'def_st_500_549_yrd_alw_ff',
                    136: 'def_st_550+_yrd_alw_ff',
                    198: 'fg_made_50_59_ff'
}

scoring_dict = {'pass_5_yrd':     0.1,
                'pass_comp':      0.4,
                'pass_incomp':     -0.2,
                'pass_td':     6,
                'pass_50_yrd_td':   3,
                'pass_int':     -2,
                'pass_2pt_con':     2,
                'pass_yrd_300_399':    3,
                'P400':    5,
                'rush_5_yrd':     0.6,
                'rush_td':     6,
                'RTD50':   3,
                'rush_2pt_con':     2,
                'rush_yrd_100_199':   3,
                'RY200':   5,
                'rec_5_yrd':    0.6,
                'receptions':     1,
                'rec_td':    6,
                'rec_50_yrd_td':  3,
                'rec_2pt_con':    2,
                'rec_yrd_100_199':  3,
                'REY200':  5,
                'pat_made':     1,
                'pat_miss':    -1,
                'fg_made_0_39':     3,
                'fg_made_40_49':    4,
                'fg_miss_0_39':    -2,
                'fg_miss_40_49':   -1,
                'fg_made_50':    5,
                'FG60':    5,
                'def_st_kick_ret_td':    6,
                'def_st_punt_ret_td':    6,
                'def_st_int_td':   5,
                'def_st_fum_ret_td':    5,
                'def_st_blk_td': 6,
                'def_st_sack':      1,
                'def_st_blk_kick':    2,
                'def_st_int':     3,
                'def_st_fum':      3,
                'def_st_safety':      2,
                'def_st_0_pts_alw':     10,
                'def_st_1_6_pts_alw':     7,
                'def_st_7_13_pts_alw':     3,
                'def_st_14_17_pts_alw':    1,
                'def_st_22_27_pts_alw':    -1,
                'def_st_28_34_pts_alw':    -3,
                'def_st_35_45_pts_alw':    -5,
                'PA46':    -7,
                'def_st_0_99_yrd_alw':   7,
                'def_st_100_199_yrd_alw':   3,
                'def_st_200_299_yrd_alw':   1,
                'def_st_400_449_yrd_alw':   -1,
                'def_st_450_499_yrd_alw':   -1.5,
                'def_st_500_549_yrd_alw':   -2,
                'def_st_550+_yrd_alw':   -3,
                'misc_kick_ret_td':    6,
                'misc_punt_ret_td':    6,
                'misc_fum_rec_td':     6,
                'misc_fum_lost':    -2,
                'misc_fum_ret_td':    6
}

#### Create Rosters Dataframe per Team per Week

In [191]:
# initialize list needed to create rosters_df
data_list = []

# loop through each JSON object in matchups_list which represents one week's matchup data
for wk in range(0, len(matchups_list)):
    #print(wk)
    
    # grab year
    year = matchups_list[wk]['seasonId']
    #print(year)
    
    # loop through each team
    for tm in matchups_list[wk]['teams']:
        owner_team_id   = tm['id']
        owner_team_name = owner_team_codes[owner_team_id][0]
        owner_name      = owner_team_codes[owner_team_id][1]
        #print(owner_name)
    
        # loop through weekly roster
        for p in tm['roster']['entries']:
            #temp_week = wk + 1
            temp_week = matchups_list[wk]['scoringPeriodId']
            #print(temp_week)
            
            # extract roster data
            player_name   = p['playerPoolEntry']['player']['fullName']
            slot_id       = p['lineupSlotId']
            slot_name     = lineup_slot_codes[slot_id]
            position_id   = p['playerPoolEntry']['player']['defaultPositionId']
            position_name = position_codes[position_id]
            #pro_team_id   = p['playerPoolEntry']['player']['proTeamId']
            #pro_team_name = pro_team_codes[pro_team_id][0]
            #pro_team_name_abv = pro_team_codes[pro_team_id][1]
            #if year != 2018:
            #    pro_team_id   = p['playerPoolEntry']['player']['proTeamId']
            #    pro_team_name = pro_team_codes[pro_team_id][0]
            #    pro_team_name_abv = pro_team_codes[pro_team_id][1]
                
            #print(player_name)

            # injured status (need try/exc bc of D/ST)
            current_inj = np.nan
            try:
                current_inj = p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass

            # projected/actual points
            proj_points, actual_points = None, None
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != temp_week:
                    continue
                if stat['statSourceId'] == 0:
                    actual_points = stat['appliedTotal']
                    #print(actual_points)
                    #print(stat['seasonId'])
                    #print(stat['proTeamId'])
                    if stat['proTeamId'] != 0:
                        #print('sucess')
                        pro_team_id = stat['proTeamId']
                        #print(pro_team_id)
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        #print(pro_team_name)
                        pro_team_name_abv = pro_team_codes[pro_team_id][1]                        
                elif stat['statSourceId'] == 1:
                    proj_points = stat['appliedTotal']
                    #print(proj_points)
                    if stat['proTeamId'] != 0:
                        #print('sucess')
                        pro_team_id = stat['proTeamId']
                        #print(pro_team_id)
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        #print(pro_team_name)
                        pro_team_name_abv = pro_team_codes[pro_team_id][1]
                    elif (proj_points < 1) & (not actual_points):
                        pro_team_id   = p['playerPoolEntry']['player']['proTeamId']
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        pro_team_name_abv = pro_team_codes[pro_team_id][1] 
                    elif not pro_team_id:
                        pro_team_id   = p['playerPoolEntry']['player']['proTeamId']
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        pro_team_name_abv = pro_team_codes[pro_team_id][1]                       

            data_list.append([year, temp_week, owner_team_name, owner_name, player_name, pro_team_name, pro_team_name_abv, 
                              current_inj, slot_name, position_name, proj_points, actual_points, slot_id])

# create rosters_df using data_list
rosters_df = pd.DataFrame(data_list, 
                          columns=['year', 'week', 'owner_team', 'owner', 'player', 'pro_team', 'pro_team_abv',
                                   'current_inj_status', 'lineup_slot_name', 'position_name', 'proj_points', 
                                   'actual_points', 'slot_id'
                                  ]
                         )

# save list of column names for later use
rosters_df_columns = rosters_df.columns.tolist()

# save to csv
rosters_df.to_csv("rosters_df.csv", index = False)

# explore data frame
rosters_df.info()
rosters_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10176 entries, 0 to 10175
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                10176 non-null  int64  
 1   week                10176 non-null  int64  
 2   owner_team          10176 non-null  object 
 3   owner               10176 non-null  object 
 4   player              10176 non-null  object 
 5   pro_team            10176 non-null  object 
 6   pro_team_abv        10157 non-null  object 
 7   current_inj_status  10176 non-null  object 
 8   lineup_slot_name    10176 non-null  object 
 9   position_name       10176 non-null  object 
 10  proj_points         10134 non-null  float64
 11  actual_points       9607 non-null   float64
 12  slot_id             10176 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 1.0+ MB


,year,week,proj_points,actual_points,slot_id
count,10176.000000,10176.000000,10134.000000,9607.000000,10176.000000
mean,2019.001474,8.991057,10.797382,11.648215,13.418239
std,0.816355,4.898460,7.271484,10.705489,8.044007
min,2018.000000,1.000000,0.000000,-8.100000,0.000000
25%,2018.000000,5.000000,6.659093,3.400000,4.000000
50%,2019.000000,9.000000,10.077620,9.000000,20.000000
75%,2020.000000,13.000000,14.768187,17.000000,20.000000
max,2020.000000,17.000000,35.516674,70.800000,20.000000


In [192]:
rosters_df.head()

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id
0,2018,1,Happy Rock Homewreckers,Blainer,David Johnson,Arizona Cardinals,ARI,ACTIVE,RB,RB,21.371488,15.8,2
1,2018,1,Happy Rock Homewreckers,Blainer,Melvin Gordon,Los Angeles Chargers,LAC,ACTIVE,RB,RB,16.624250,27.8,2
2,2018,1,Happy Rock Homewreckers,Blainer,Rob Gronkowski,New Engalnd Patriots,NE,ACTIVE,TE,TE,14.862558,24.2,6
3,2018,1,Happy Rock Homewreckers,Blainer,Allen Robinson,Chicago Bears,CHI,ACTIVE,WR,WR,13.913777,8.8,4
4,2018,1,Happy Rock Homewreckers,Blainer,Marvin Jones Jr.,Detroit Lions,DET,INJURY_RESERVE,WR,WR,11.560917,7.6,4


In [193]:
rosters_df.tail()

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id
10171,2020,17,Bud Lathrop Drive,Farmer,Salvon Ahmed,Miami Dolphins,MIA,ACTIVE,Bench,RB,6.514888,13.8,20
10172,2020,17,Bud Lathrop Drive,Farmer,Browns D/ST,Cleveland Browns,CLE,NA,Bench,DEF,9.101029,3.0,20
10173,2020,17,Bud Lathrop Drive,Farmer,Gus Edwards,Baltimore Ravens,BAL,ACTIVE,Bench,RB,10.006076,9.4,20
10174,2020,17,Bud Lathrop Drive,Farmer,David Johnson,Houston Texans,HOU,ACTIVE,Flex,RB,16.786423,22.8,3
10175,2020,17,Bud Lathrop Drive,Farmer,Tre'Quan Smith,New Orleans Saints,NO,ACTIVE,WR,WR,0.000000,0.0,4


In [194]:
rosters_df[rosters_df['year'] == 2018].tail()

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id
3378,2018,17,Bud Lathrop Drive,Farmer,Joe Flacco,Baltimore Ravens,BAL,ACTIVE,Bench,QB,0.457230,0.0,20
3379,2018,17,Bud Lathrop Drive,Farmer,Baker Mayfield,Cleveland Browns,CLE,ACTIVE,Bench,QB,20.247639,27.9,20
3380,2018,17,Bud Lathrop Drive,Farmer,Jake Elliott,Philadelphia Eagles,PHI,ACTIVE,K,KR,7.578694,6.0,17
3381,2018,17,Bud Lathrop Drive,Farmer,Gus Edwards,Baltimore Ravens,BAL,ACTIVE,RB,RB,12.136761,9.0,2
3382,2018,17,Bud Lathrop Drive,Farmer,49ers D/ST,San Francisco 49ers,SF,NA,Bench,DEF,6.824698,0.0,20


In [200]:
rosters_df[(rosters_df['year'] == 2018) & (rosters_df['player'].str.contains('Tate'))]

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id
23,2018,1,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,Flex,WR,10.926680,17.8,3
223,2018,2,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,Flex,WR,12.246540,19.0,3
423,2018,3,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,Flex,WR,12.371472,10.2,3
623,2018,4,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,Bench,WR,13.038936,33.8,20
821,2018,5,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,WR,WR,14.893074,6.8,4
1021,2018,6,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,Bench,WR,0.031522,NaN,20
1221,2018,7,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,WR,WR,13.508474,9.4,4
1421,2018,8,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,WR,WR,12.111631,8.8,4
1621,2018,9,Bench Don't Kill My Vibe,Padge,Golden Tate,Detroit Lions,DET,ACTIVE,Bench,WR,0.007880,NaN,20
1819,2018,10,Bench Don't Kill My Vibe,Padge,Golden Tate,Philadelphia Eagles,PHI,ACTIVE,Flex,WR,11.613705,2.6,3


In [198]:
rosters_df[(rosters_df['year'] == 2019) & (rosters_df['player'].str.contains('Mahomes'))]

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id
3463,2019,1,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,26.821497,39.9,0
3663,2019,2,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,33.347753,45.0,0
3863,2019,3,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,30.907860,40.8,0
4063,2019,4,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,33.907936,21.3,0
4263,2019,5,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,31.901909,22.6,0
4463,2019,6,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,30.641790,23.8,0
4663,2019,7,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,28.478373,11.3,0
4863,2019,8,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,Bench,QB,0.000000,0.0,20
5063,2019,9,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,Bench,QB,0.000000,0.0,20
5263,2019,10,Springfield Atoms,Duvi,Patrick Mahomes,Kansas City Chiefs,KC,ACTIVE,QB,QB,30.590295,46.5,0


In [199]:
rosters_df[rosters_df['pro_team'] == 'Free Agent']

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id
2669,2018,14,Springfield Atoms,Duvi,Kareem Hunt,Free Agent,NaN,NA,Bench,RB,0.0,NaN,20
2867,2018,15,Springfield Atoms,Duvi,Kareem Hunt,Free Agent,NaN,NA,Bench,RB,0.0,NaN,20
3065,2018,16,Springfield Atoms,Duvi,Kareem Hunt,Free Agent,NaN,NA,Bench,RB,0.0,NaN,20
3263,2018,17,Springfield Atoms,Duvi,Kareem Hunt,Free Agent,NaN,NA,Bench,RB,0.0,NaN,20
3785,2019,3,Happy Rock Homewreckers,Blainer,Antonio Brown,Free Agent,NaN,ACTIVE,Bench,WR,0.0,NaN,20
3985,2019,4,Happy Rock Homewreckers,Blainer,Antonio Brown,Free Agent,NaN,ACTIVE,Bench,WR,0.0,NaN,20
6969,2020,1,Bud Lathrop Drive,Farmer,Golden Tate,Free Agent,NaN,QUESTIONABLE,Bench,WR,0.0,0.0,20
6992,2020,2,Happy Rock Homewreckers,Blainer,Duke Johnson,Free Agent,NaN,QUESTIONABLE,Bench,RB,0.0,0.0,20
7476,2020,4,Springfield Atoms,Duvi,Adam Humphries,Free Agent,NaN,QUESTIONABLE,Bench,WR,0.0,NaN,20
7529,2020,4,CoMo FightinCamlToes,Doisy,Jared Cook,Free Agent,NaN,ACTIVE,Bench,TE,0.0,0.0,20


In [188]:
print(matchups_list[50]['seasonId'])
print(matchups_list[50]['scoringPeriodId'])
print(owner_team_codes[matchups_list[50]['teams'][4]['id']][1])
print(matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['fullName'])
print(matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['proTeamId'])
print(pro_team_codes[matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['proTeamId']][0])
print(pro_team_codes[matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['proTeamId']][1])
print('--------------')
print(pro_team_codes[matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['stats'][0]['proTeamId']][0])
print(pro_team_codes[matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['stats'][0]['proTeamId']][1])

print(matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['stats'][0]['scoringPeriodId'])
print(matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['stats'][0]['statSourceId'])
print(matchups_list[50]['teams'][4]['roster']['entries'][4]['playerPoolEntry']['player']['stats'][0]['proTeamId'])

2020
17
Duvi
Mark Ingram II
0
Free Agent
nan
--------------
Baltimore Ravens
BAL
17
0
33


In [189]:
print(matchups_list[50]['seasonId'])
print(matchups_list[50]['scoringPeriodId'])
matchups_list[50]['teams'][4]['roster']['entries'][4]

2020
17


{'acquisitionDate': None,
 'acquisitionType': None,
 'injuryStatus': 'NORMAL',
 'lineupSlotId': 20,
 'pendingTransactionIds': None,
 'playerId': 13981,
 'playerPoolEntry': {'appliedStatTotal': 4.2,
  'id': 13981,
  'keeperValue': 4,
  'keeperValueFuture': 7,
  'lineupLocked': True,
  'onTeamId': 7,
  'player': {'active': True,
   'defaultPositionId': 2,
   'droppable': True,
   'eligibleSlots': [2, 3, 23, 7, 20, 21],
   'firstName': 'Mark',
   'fullName': 'Mark Ingram II',
   'id': 13981,
   'injured': False,
   'injuryStatus': 'ACTIVE',
   'lastName': 'Ingram II',
   'lastNewsDate': 1611090821000,
   'proTeamId': 0,
   'rankings': {'0': [{'auctionValue': 0,
      'rank': 29,
      'rankSourceId': 7,
      'rankType': 'STANDARD',
      'slotId': 2},
     {'auctionValue': 0,
      'rank': 30,
      'rankSourceId': 7,
      'rankType': 'PPR',
      'slotId': 2},
     {'auctionValue': 0,
      'rank': 21,
      'rankSourceId': 5,
      'rankType': 'STANDARD',
      'slotId': 2},
     {'au

#### Add Fantasy Scoring Data to Rosters Dataframe

In [201]:
# create columns using fantasy scoring values and fill each with 0
for i in [str(s) for s in ff_scoring_codes.values()]:
    rosters_df[i] = 0
    
# create columns using player stat values and fill each with 0
for j in [str(r) for r in player_stat_codes.values()]:
    rosters_df[j] = 0

In [202]:
# loop through each JSON object in matchups_list which represents one week's matchup data
for wk in range(0, len(matchups_list)):
    
    # grab year
    year = matchups_list[wk]['seasonId']

    # loop through each team
    for tm in matchups_list[wk]['teams']:

        # loop through weekly roster
        for p in tm['roster']['entries']:
            #temp_week = wk + 1
            temp_week = matchups_list[wk]['scoringPeriodId']
            
            # grab player name
            player_name = p['playerPoolEntry']['player']['fullName']

            # loop through each set of stats
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != temp_week:
                    continue
                if stat['statSourceId'] == 0:
                    
                    # loop through the fantasy scoring stats
                    for i in [int(s) for s in stat['appliedStats'].keys()]:

                        # if the scoring code exists in the dictionary above then add the stat to rosters_df
                        if i in ff_scoring_codes.keys():
                            rosters_df.loc[(rosters_df['player'] == player_name) & (rosters_df['week'] == temp_week) &\
                                           (rosters_df['year'] == year), ff_scoring_codes[i]] = stat['appliedStats'][str(i)]

                    # loop through the player stats
                    for j in [int(r) for r in stat['stats'].keys()]:

                        # if the scoring code exists in the dictionary above then add the stat to rosters_df
                        if j in player_stat_codes.keys():
                            rosters_df.loc[(rosters_df['player'] == player_name) & (rosters_df['week'] == temp_week) &\
                                           (rosters_df['year'] == year), player_stat_codes[j]] = stat['stats'][str(j)]

# replace nulls with 0
rosters_df.replace(np.nan, 0, inplace=True)

# save to csv
rosters_df.to_csv("rosters_df_w_scoring.csv", index = False)

# explore data frame
rosters_df.info()
rosters_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10176 entries, 0 to 10175
Columns: 236 entries, year to unk210
dtypes: float64(223), int64(5), object(8)
memory usage: 18.3+ MB


,year,week,proj_points,actual_points,slot_id,pass_comp_ff,pass_incomp_ff,pass_td_ff,pass_5_yrd_ff,pass_50_yrd_td_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_2pt_con_ff,pass_int_ff,rush_td_ff,rush_2pt_con_ff,rush_5_yrd_ff,rush_50_yrd_td_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rec_td_ff,rec_2pt_con_ff_ff,rec_50_yrd_td_ff,rec_5_yrd_ff,receptions_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,fum_lost_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_0_39_ff,fg_miss_0_39_ff,pat_made_ff,pat_miss_ff,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_blk_td_ff,def_st_int_ff,def_st_fum_ff,def_st_blk_kick_ff,def_st_safety_ff,def_st_sack_ff,def_st_kick_ret_td_ff,def_st_punt_ret_td_ff,def_st_int_td_ff,def_st_fum_ret_td_ff,def_st_22_27_pts_alw_ff,def_st_28_34_pts_alw_ff,def_st_35_45_pts_alw_ff,def_st_46+_pts_alw_ff,def_st_0_99_yrd_alw_ff,def_st_100_199_yrd_alw_ff,def_st_200_299_yrd_alw_ff,def_st_350_399_yrd_alw_ff,def_st_400_449_yrd_alw_ff,def_st_450_499_yrd_alw_ff,def_st_500_549_yrd_alw_ff,def_st_550+_yrd_alw_ff,fg_made_50_59_ff,pass_att,pass_comp,pass_incomp,pass_yrd,pass_td,pass_5_yrd,unk6,unk7,unk8,unk9,unk10,unk11,unk12,unk13,unk14,unk15,pass_50_yrd_td,pass_yrd_300_399,pass_yrd_400+,pass_2pt_con,pass_int,unk21,pass_yrd_dupe,rush_att,rush_yrd,rush_td,rush_2pt_con,rush_5_yrd,unk28,unk29,unk30,unk31,unk32,unk33,unk34,unk35,rush_50_yrd_td,rush_yrd_100_199,rush_yrd_200+,unk39,unk40,receptions_dupe,rec_yrd,rec_td,rec_2pt_con,unk45,rec_50_yrd_td,rec_5_yrd,unk48,unk49,unk50,unk51,unk52,receptions,unk54,unk55,rec_yrd_100_199,rec_yrd_200+,rec_tar,yac,yrd_per_rec,rec_yrd_dupe,unk62,unk64,unk65,unk66,unk67,unk68,unk69,unk70,unk71,fum_lost,unk73,fg_made_50+,unk75,unk76,fg_made_40_49,unk78,fg_miss_40_49,fg_made_0_39,unk81,fg_miss_0_39,fg_con,fg_att,fg_miss_tot,pat_con,pat_att,pat_miss_tot,def_st_0_pts_alw,def_st_1_6_pts_alw,def_st_7_13_pts_alw,def_st_14_17_pts_alw,def_st_blk_td,unk94,def_st_int,def_st_fum,def_st_blk_kick,def_st_safety,def_st_sack,unk100,def_st_kick_ret_td,def_st_punt_ret_td,def_st_int_td,def_st_fum_ret_td,unk105,unk106,unk107,unk108,unk109,unk110,unk111,unk112,unk113,unk114,unk115,unk116,unk117,unk118,unk119,def_pts_alw,unk121,def_st_22_27_pts_alw,def_st_28_34_pts_alw,def_st_35_45_pts_alw,def_st_46+_pts_alw,def_tot_yrd_alw,def_st_0_99_yrd_alw,def_st_100_199_yrd_alw,def_st_200_299_yrd_alw,unk131,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw,def_st_450_499_yrd_alw,def_st_500_549_yrd_alw,def_st_550+_yrd_alw,unk155,unk156,unk158,unk175,unk176,unk177,unk178,unk179,unk180,unk181,unk182,unk183,unk184,unk185,unk186,unk187,unk188,unk189,unk190,unk191,unk192,unk193,unk194,unk195,unk196,unk197,fg_made_50_59,unk199,unk200,unk202,unk203,unk210
count,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.00000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.00000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.0,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.00000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.000000,10176.00

In [203]:
rosters_df.head(10)

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id,pass_comp_ff,pass_incomp_ff,pass_td_ff,pass_5_yrd_ff,pass_50_yrd_td_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_2pt_con_ff,pass_int_ff,rush_td_ff,rush_2pt_con_ff,rush_5_yrd_ff,rush_50_yrd_td_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rec_td_ff,rec_2pt_con_ff_ff,rec_50_yrd_td_ff,rec_5_yrd_ff,receptions_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,fum_lost_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_0_39_ff,fg_miss_0_39_ff,pat_made_ff,pat_miss_ff,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_blk_td_ff,def_st_int_ff,def_st_fum_ff,def_st_blk_kick_ff,def_st_safety_ff,def_st_sack_ff,def_st_kick_ret_td_ff,def_st_punt_ret_td_ff,def_st_int_td_ff,def_st_fum_ret_td_ff,def_st_22_27_pts_alw_ff,def_st_28_34_pts_alw_ff,def_st_35_45_pts_alw_ff,def_st_46+_pts_alw_ff,def_st_0_99_yrd_alw_ff,def_st_100_199_yrd_alw_ff,def_st_200_299_yrd_alw_ff,def_st_350_399_yrd_alw_ff,def_st_400_449_yrd_alw_ff,def_st_450_499_yrd_alw_ff,def_st_500_549_yrd_alw_ff,def_st_550+_yrd_alw_ff,fg_made_50_59_ff,pass_att,pass_comp,pass_incomp,pass_yrd,pass_td,pass_5_yrd,unk6,unk7,unk8,unk9,unk10,unk11,unk12,unk13,unk14,unk15,pass_50_yrd_td,pass_yrd_300_399,pass_yrd_400+,pass_2pt_con,pass_int,unk21,pass_yrd_dupe,rush_att,rush_yrd,rush_td,rush_2pt_con,rush_5_yrd,unk28,unk29,unk30,unk31,unk32,unk33,unk34,unk35,rush_50_yrd_td,rush_yrd_100_199,rush_yrd_200+,unk39,unk40,receptions_dupe,rec_yrd,rec_td,rec_2pt_con,unk45,rec_50_yrd_td,rec_5_yrd,unk48,unk49,unk50,unk51,unk52,receptions,unk54,unk55,rec_yrd_100_199,rec_yrd_200+,rec_tar,yac,yrd_per_rec,rec_yrd_dupe,unk62,unk64,unk65,unk66,unk67,unk68,unk69,unk70,unk71,fum_lost,unk73,fg_made_50+,unk75,unk76,fg_made_40_49,unk78,fg_miss_40_49,fg_made_0_39,unk81,fg_miss_0_39,fg_con,fg_att,fg_miss_tot,pat_con,pat_att,pat_miss_tot,def_st_0_pts_alw,def_st_1_6_pts_alw,def_st_7_13_pts_alw,def_st_14_17_pts_alw,def_st_blk_td,unk94,def_st_int,def_st_fum,def_st_blk_kick,def_st_safety,def_st_sack,unk100,def_st_kick_ret_td,def_st_punt_ret_td,def_st_int_td,def_st_fum_ret_td,unk105,unk106,unk107,unk108,unk109,unk110,unk111,unk112,unk113,unk114,unk115,unk116,unk117,unk118,unk119,def_pts_alw,unk121,def_st_22_27_pts_alw,def_st_28_34_pts_alw,def_st_35_45_pts_alw,def_st_46+_pts_alw,def_tot_yrd_alw,def_st_0_99_yrd_alw,def_st_100_199_yrd_alw,def_st_200_299_yrd_alw,unk131,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw,def_st_450_499_yrd_alw,def_st_500_549_yrd_alw,def_st_550+_yrd_alw,unk155,unk156,unk158,unk175,unk176,unk177,unk178,unk179,unk180,unk181,unk182,unk183,unk184,unk185,unk186,unk187,unk188,unk189,unk190,unk191,unk192,unk193,unk194,unk195,unk196,unk197,fg_made_50_59,unk199,unk200,unk202,unk203,unk210
0,2018,1,Happy Rock Homewreckers,Blainer,David Johnson,Arizona Cardinals,ARI,ACTIVE,RB,RB,21.371488,15.8,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,4.2,0.0,0.0,0.0,0.0,0.0,0.0,3.6,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,9.0,37.0,1.0,0.0,7.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.111111,37.0,5.0,30.0,0.0,0.0,0.0,0.0,6.0,3.0,1.0,1.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,9.0,0.0,6.000000,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018,1,Happy Rock Homewreckers,Blainer,Melvin Gordon,Los Angeles Chargers,LAC,ACTIVE,RB,RB,16.624250,27.8,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,1

In [204]:
rosters_df.tail(20)

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id,pass_comp_ff,pass_incomp_ff,pass_td_ff,pass_5_yrd_ff,pass_50_yrd_td_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_2pt_con_ff,pass_int_ff,rush_td_ff,rush_2pt_con_ff,rush_5_yrd_ff,rush_50_yrd_td_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rec_td_ff,rec_2pt_con_ff_ff,rec_50_yrd_td_ff,rec_5_yrd_ff,receptions_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,fum_lost_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_0_39_ff,fg_miss_0_39_ff,pat_made_ff,pat_miss_ff,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_blk_td_ff,def_st_int_ff,def_st_fum_ff,def_st_blk_kick_ff,def_st_safety_ff,def_st_sack_ff,def_st_kick_ret_td_ff,def_st_punt_ret_td_ff,def_st_int_td_ff,def_st_fum_ret_td_ff,def_st_22_27_pts_alw_ff,def_st_28_34_pts_alw_ff,def_st_35_45_pts_alw_ff,def_st_46+_pts_alw_ff,def_st_0_99_yrd_alw_ff,def_st_100_199_yrd_alw_ff,def_st_200_299_yrd_alw_ff,def_st_350_399_yrd_alw_ff,def_st_400_449_yrd_alw_ff,def_st_450_499_yrd_alw_ff,def_st_500_549_yrd_alw_ff,def_st_550+_yrd_alw_ff,fg_made_50_59_ff,pass_att,pass_comp,pass_incomp,pass_yrd,pass_td,pass_5_yrd,unk6,unk7,unk8,unk9,unk10,unk11,unk12,unk13,unk14,unk15,pass_50_yrd_td,pass_yrd_300_399,pass_yrd_400+,pass_2pt_con,pass_int,unk21,pass_yrd_dupe,rush_att,rush_yrd,rush_td,rush_2pt_con,rush_5_yrd,unk28,unk29,unk30,unk31,unk32,unk33,unk34,unk35,rush_50_yrd_td,rush_yrd_100_199,rush_yrd_200+,unk39,unk40,receptions_dupe,rec_yrd,rec_td,rec_2pt_con,unk45,rec_50_yrd_td,rec_5_yrd,unk48,unk49,unk50,unk51,unk52,receptions,unk54,unk55,rec_yrd_100_199,rec_yrd_200+,rec_tar,yac,yrd_per_rec,rec_yrd_dupe,unk62,unk64,unk65,unk66,unk67,unk68,unk69,unk70,unk71,fum_lost,unk73,fg_made_50+,unk75,unk76,fg_made_40_49,unk78,fg_miss_40_49,fg_made_0_39,unk81,fg_miss_0_39,fg_con,fg_att,fg_miss_tot,pat_con,pat_att,pat_miss_tot,def_st_0_pts_alw,def_st_1_6_pts_alw,def_st_7_13_pts_alw,def_st_14_17_pts_alw,def_st_blk_td,unk94,def_st_int,def_st_fum,def_st_blk_kick,def_st_safety,def_st_sack,unk100,def_st_kick_ret_td,def_st_punt_ret_td,def_st_int_td,def_st_fum_ret_td,unk105,unk106,unk107,unk108,unk109,unk110,unk111,unk112,unk113,unk114,unk115,unk116,unk117,unk118,unk119,def_pts_alw,unk121,def_st_22_27_pts_alw,def_st_28_34_pts_alw,def_st_35_45_pts_alw,def_st_46+_pts_alw,def_tot_yrd_alw,def_st_0_99_yrd_alw,def_st_100_199_yrd_alw,def_st_200_299_yrd_alw,unk131,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw,def_st_450_499_yrd_alw,def_st_500_549_yrd_alw,def_st_550+_yrd_alw,unk155,unk156,unk158,unk175,unk176,unk177,unk178,unk179,unk180,unk181,unk182,unk183,unk184,unk185,unk186,unk187,unk188,unk189,unk190,unk191,unk192,unk193,unk194,unk195,unk196,unk197,fg_made_50_59,unk199,unk200,unk202,unk203,unk210
10156,2020,17,Bud Lathrop Drive,Farmer,Michael Thomas,New Orleans Saints,NO,QUESTIONABLE,Bench,WR,0.000000,0.0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
10157,2020,17,Bud Lathrop Drive,Farmer,Austin Ekeler,Los Angeles Chargers,LAC,QUESTIONABLE,RB,RB,20.422703,17.4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,6.0,0.0,0.0,3.6

In [ ]:
print(matchups_list[0]['seasonId'])
print(matchups_list[0]['scoringPeriodId'])
print(owner_team_codes[matchups_list[0]['teams'][7]['id']][1])
print(matchups_list[0]['teams'][7]['roster']['entries'][19]['playerPoolEntry']['player']['fullName'])
print(matchups_list[0]['teams'][7]['roster']['entries'][19]['playerPoolEntry']['player']['proTeamId'])
print(pro_team_codes[matchups_list[5]['teams'][1]['roster']['entries'][2]['playerPoolEntry']['player']['proTeamId']][0])
print(pro_team_codes[matchups_list[5]['teams'][1]['roster']['entries'][2]['playerPoolEntry']['player']['proTeamId']][1])

In [ ]:
for stat in matchups_list[1]['teams'][1]['roster']['entries'][3]['playerPoolEntry']['player']['stats']:
    if stat['scoringPeriodId'] != 1:
        continue
    if stat['statSourceId'] == 0:
        print(stat['appliedStats'])
        print(stat['proTeamId'])

In [ ]:
print(matchups_list[0]['seasonId'])
print(matchups_list[0]['scoringPeriodId'])
matchups_list[0]['teams'][7]['roster']['entries'][19]

#### Create Matchups Dataframe per Team per Week

In [ ]:
# initialize list needed to create matchups_df
data_list = []

# loop through each matchup from week 1 to current week
for i in range(0, len(data['schedule'])):
    
    if data['schedule'][i]['away']['totalPoints'] == 0:
        break
        
    # build row for away team
    away_week = data['schedule'][i]['matchupPeriodId']
    away_owner_team_id = data['schedule'][i]['away']['teamId']
    away_owner_team_name = owner_team_codes[away_owner_team_id][0]
    away_owner_name = owner_team_codes[away_owner_team_id][1]
    away_score = data['schedule'][i]['away']['totalPoints']
    away_opp_id = data['schedule'][i]['home']['teamId']
    away_opp_team_name = owner_team_codes[away_opp_id][0]
    away_opp_name = owner_team_codes[away_opp_id][1]
    away_opp_score = data['schedule'][i]['home']['totalPoints']
    
    # determine if away team won
    if data['schedule'][i]['winner'] == 'AWAY':
        away_win = 1
    else:
        away_win = 0
    # append away row to data_list
    data_list.append([away_week, away_owner_team_name, away_owner_name, away_score, away_win, away_opp_team_name, 
                      away_opp_name, away_opp_score])
    
    # build row for home team
    home_week = data['schedule'][i]['matchupPeriodId']
    home_owner_team_id = data['schedule'][i]['home']['teamId']
    home_owner_team_name = owner_team_codes[home_owner_team_id][0]
    home_owner_name = owner_team_codes[home_owner_team_id][1]
    home_score = data['schedule'][i]['home']['totalPoints']
    home_opp_id = data['schedule'][i]['away']['teamId']
    home_opp_team_name = owner_team_codes[home_opp_id][0]
    home_opp_name = owner_team_codes[home_opp_id][1]
    home_opp_score = data['schedule'][i]['away']['totalPoints']
    
    # determine if home team won    
    if data['schedule'][i]['winner'] == 'HOME':
        home_win = 1
    else:
        home_win = 0
    
    # append home row to data_list
    data_list.append([home_week, home_owner_team_name, home_owner_name, home_score, home_win, home_opp_team_name, 
                      home_opp_name, home_opp_score])

# create matchups_df using data_list
matchups_df = pd.DataFrame(data_list, 
                          columns=['week', 'owner_team_name', 'owner', 'score', 'win', 'opp_owner_team_name', 'opp_owner', 
                                   'opp_score'])
# save to csv
matchups_df.to_csv("matchups_df.csv", index = False)

# explore data frame
matchups_df.info()
matchups_df.describe()

In [ ]:
# subset matchups_df by wins
wins = matchups_df.loc[matchups_df['win'] == 1]

# create total_wins dataframe of wins per team
total_wins = pd.DataFrame(wins.groupby(['owner_team_name'])['win'].value_counts().reset_index(0).reset_index(drop=True))
total_wins.columns = ['owner_team_name', 'wins']

# subset matchups_df by losses
losses = matchups_df.loc[matchups_df['win'] == 0]

# create total_losses dataframe of losses per team
total_losses = pd.DataFrame(losses.groupby(['owner_team_name'])['win'].value_counts().reset_index(0).reset_index(drop=True))
total_losses.columns = ['owner_team_name', 'losses']

# merge total_wins and total_losses
win_loss_df = total_wins.merge(total_losses, on = 'owner_team_name', how = 'left')

# replace any null values with 0 which means one or more teams have either 0 wins or 0 losses
win_loss_df.fillna(0, inplace=True)

# fillna function casts dtype to float so change dtype back to int
win_loss_df['losses'] = win_loss_df['losses'].astype('int')
win_loss_df['wins'] = win_loss_df['wins'].astype('int')

# # create total_points dataframe of wins per team
total_points_df = pd.DataFrame(matchups_df.groupby(['owner_team_name'])[['score', 
                                                                     'opp_score']].sum().reset_index(0).reset_index(drop=True))
total_points_df.columns = ['owner_team_name', 'points_for', 'points_against']

# merge win_loss_df with total_points_df
win_loss_df = win_loss_df.merge(total_points_df, on = 'owner_team_name', how = 'left')

# save to csv
win_loss_df.to_csv("win_loss_df.csv", index = False)

# explore data frame
win_loss_df.info()
win_loss_df.describe()

In [ ]:
win_loss_df